In [241]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import random
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [242]:
DATADIR=r"C:\Users\USER\Desktop\our project\old\Alzheimer_s Dataset\train"
CATEGORIES=["MildDemented","ModerateDemented","NonDemented","VeryMildDemented"]

In [243]:
training_data=[]
def create_training_data():
    for category in CATEGORIES:  
        path = os.path.join(DATADIR,category)  # create path 
        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1 or 2 or 3). 0=MildDemented 1=ModerateDemented 2=NonDemented 3=VeryMildDemented
        for img1 in os.listdir(path):  # iterate over each image 
      
                img = cv2.imread(os.path.join(path,img1))  # convert to array
    
                img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
                #plt.imshow(img)
                # cv2.imshow("img1", img1)
                # cv2.waitKey(0)
                # cv2.destroyAllWindows()
                #print (img.shape)
                th = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
                ret,th = cv2.threshold(th, 110, 255, cv2.THRESH_BINARY)
                kernel = np.ones_like((5,5),np.uint8)
                erosion = cv2.erode(th,kernel,iterations = 1)
                # dilation = cv2.dilate(erosion,kernel,iterations = 1)
                #plt.imshow(erosion, 'gray')
    
                cont,_= cv2.findContours(erosion, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
                # find the biggest countour (c) by the area
                c = max(cont, key = cv2.contourArea)
                x,y,w,h = cv2.boundingRect(c)
    
                # draw the biggest contour (c) in green
                cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,0),0) 
                #plt.imshow(img)
                cropped_imge= img[y:y+h, x:x+w]
                #plt.imshow(cropped_imge, 'gray')
                IMG_SIZE_y = 100
                IMG_SIZE_x = 85
                resiz = cv2.resize(cropped_imge,(IMG_SIZE_x,IMG_SIZE_y))
                training_data.append([resiz, class_num])
        
        #plt.imshow(resiz,'gray')
        #print (resiz.shape)
                
        #plt.show()  # display!
       # break  # we just want one for now so break
    #break

        
create_training_data()
  

In [244]:

print(len(training_data))

5133


In [245]:
random.shuffle(training_data)

In [246]:
for sample in training_data[:1]:
    print(sample[1]) #sample[0] would be the image

3


In [247]:
x = []
y = []
for features,label in training_data:
    x.append(features)
    y.append(label)
x = np.array(x)
y = np.array(y)
# x = tf.stack(x)
# y = tf.stack(y)
print(x.shape)
print(y.shape)


(5133, 100, 85, 3)
(5133,)


In [248]:
def build_model():
    
    '''Sequential Model creation'''
    model = Sequential()
    model.add(Conv2D(32,(1,1),padding='same',strides=2,input_shape = (100, 85, 3),activation='relu'))
    
    model.add(Conv2D(64,(1,1),padding='same',strides=2,activation='relu'))
    model.add(MaxPooling2D(pool_size=(3,3),strides=1,padding = 'same'))

    model.add(Conv2D(128,(1,1),padding='valid',strides=2,activation='relu'))
    model.add(MaxPooling2D(pool_size=(3,3),strides=1,padding = 'same'))
    
    model.add(Conv2D(256,(1,1),padding='valid',strides=2,activation='relu'))
    model.add(MaxPooling2D(pool_size=(3,3),strides=1,padding = 'same'))
    
    model.add(Conv2D(512,(1,1),padding='valid',strides=2,activation='relu'))
    model.add(MaxPooling2D(pool_size=(3,3),strides=1,padding = 'valid'))
    
    model.add(Conv2D(1024,(1,1),padding='valid',strides=2,activation='relu'))
    model.add(MaxPooling2D(pool_size=(3,3),strides=1,padding = 'same'))
    
    
    model.add(Flatten())
    model.add(Dense(100))
    model.add(Dense(4))
    model.add(Activation('softmax'))
    
    return model

model = build_model()
#model.summary()

In [249]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [250]:
model.fit(x,y,epochs = 10, verbose=1,validation_split=0.3)

Epoch 1/10
113/113 [==============================] - 14s 120ms/step - loss: 6.9507 - accuracy: 0.4175 - val_loss: 1.0480 - val_accuracy: 0.5097
Epoch 2/10
113/113 [==============================] - 15s 134ms/step - loss: 1.0591 - accuracy: 0.4728 - val_loss: 1.0381 - val_accuracy: 0.5097
Epoch 3/10
113/113 [==============================] - 15s 131ms/step - loss: 1.0513 - accuracy: 0.4997 - val_loss: 1.1055 - val_accuracy: 0.5097
Epoch 4/10
113/113 [==============================] - 15s 133ms/step - loss: 1.0800 - accuracy: 0.4766 - val_loss: 1.0501 - val_accuracy: 0.5097
Epoch 5/10
113/113 [==============================] - 15s 134ms/step - loss: 1.0727 - accuracy: 0.4890 - val_loss: 1.0358 - val_accuracy: 0.5097
Epoch 6/10
113/113 [==============================] - 15s 132ms/step - loss: 1.0438 - accuracy: 0.4976 - val_loss: 1.0394 - val_accuracy: 0.5097
Epoch 7/10
113/113 [==============================] - 15s 136ms/step - loss: 1.0584 - accuracy: 0.4943 - val_loss: 1.0361 - val_ac